In [ ]:
import numpy as np

In [ ]:
txt_data = "abcdefghijklmnopqrstuvwxyz abcdefghijklmnopqrstuvwxyz abcdefghijklmnopqrstuvwxyz " # input data

In [ ]:
chars = list(set(txt_data))
num_chars = len(chars) 
txt_data_size = len(txt_data)

print("unique characters : ", num_chars) 
print("txt_data_size : ", txt_data_size)

unique characters :  27
txt_data_size :  81


In [ ]:
char_to_int = dict((c, i) for i, c in enumerate(chars)) 
int_to_char = dict((i, c) for i, c in enumerate(chars))
print(char_to_int)
print("----------------------------------------------------")
print(int_to_char)
print("----------------------------------------------------")
integer_encoded = [char_to_int[i] for i in txt_data] 
print(integer_encoded)
print("----------------------------------------------------")
print("data length : ", len(integer_encoded))

{'e': 0, 'o': 1, 'g': 2, 's': 3, 'n': 4, 'm': 5, 'p': 6, 'j': 7, ' ': 8, 'q': 9, 'i': 10, 'b': 11, 'd': 12, 'k': 13, 'x': 14, 'y': 15, 'z': 16, 'c': 17, 'v': 18, 'f': 19, 'h': 20, 'w': 21, 'a': 22, 'l': 23, 'u': 24, 't': 25, 'r': 26}
----------------------------------------------------
{0: 'e', 1: 'o', 2: 'g', 3: 's', 4: 'n', 5: 'm', 6: 'p', 7: 'j', 8: ' ', 9: 'q', 10: 'i', 11: 'b', 12: 'd', 13: 'k', 14: 'x', 15: 'y', 16: 'z', 17: 'c', 18: 'v', 19: 'f', 20: 'h', 21: 'w', 22: 'a', 23: 'l', 24: 'u', 25: 't', 26: 'r'}
----------------------------------------------------
[22, 11, 17, 12, 0, 19, 2, 20, 10, 7, 13, 23, 5, 4, 1, 6, 9, 26, 3, 25, 24, 18, 21, 14, 15, 16, 8, 22, 11, 17, 12, 0, 19, 2, 20, 10, 7, 13, 23, 5, 4, 1, 6, 9, 26, 3, 25, 24, 18, 21, 14, 15, 16, 8, 22, 11, 17, 12, 0, 19, 2, 20, 10, 7, 13, 23, 5, 4, 1, 6, 9, 26, 3, 25, 24, 18, 21, 14, 15, 16, 8]
----------------------------------------------------
data length :  81


In [ ]:
onehot_encoded = []

for ix in integer_encoded:
    letter = [0 for _ in range(len(chars))]
    letter[ix] = 1 
    onehot_encoded.append(letter) 
onehot_encoded = np.array(onehot_encoded)  

print(onehot_encoded.shape)     
print(onehot_encoded)

(81, 27)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
inverted = int_to_char[np.argmax(onehot_encoded[0])] 
print(inverted)

a


In [ ]:
iteration = 5000
sequence_length = 10
batch_size = round((txt_data_size /sequence_length)+0.5) 
hidden_size = 100   
learning_rate = 1e-1

W_xh = np.random.randn(hidden_size, num_chars)*0.01     
W_hh = np.random.randn(hidden_size, hidden_size)*0.01  
W_hy = np.random.randn(num_chars, hidden_size)*0.01    

b_h = np.zeros((hidden_size, 1)) 
b_y = np.zeros((num_chars, 1)) 

h_prev = np.zeros((hidden_size,1)) 

In [ ]:
def forwardprop(inputs, targets, h_prev):

    xs, hs, ys, ps = {}, {}, {}, {} 
    hs[-1] = np.copy(h_prev) 
    loss = 0 
    
    for t in range(len(inputs)): 
        
        xs[t] = np.zeros((num_chars,1)) 
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(W_xh, xs[t]) + np.dot(W_hh, hs[t-1]) + b_h) 
        ys[t] = np.dot(W_hy, hs[t]) + b_y 
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) 
 
        loss += -np.log(ps[t][targets[t],0])
    return loss, ps, hs, xs 

In [ ]:
def backprop(ps, inputs, hs, xs):

    dWxh, dWhh, dWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy) 
    dbh, dby = np.zeros_like(b_h), np.zeros_like(b_y)
    dhnext = np.zeros_like(hs[0]) 

    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t]) 
        dy[targets[t]] -= 1 
        dWhy += np.dot(dy, hs[t].T)
        dby += dy 
        dh = np.dot(W_hy.T, dy) + dhnext 
        dhraw = (1 - hs[t] * hs[t]) * dh
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(W_hh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]: 
        np.clip(dparam, -5, 5, out=dparam) 
    
    return dWxh, dWhh, dWhy, dbh, dby

In [ ]:
data_pointer = 0

mWxh, mWhh, mWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy)
mbh, mby = np.zeros_like(b_h), np.zeros_like(b_y) 


for i in range(iteration):
    h_prev = np.zeros((hidden_size,1)) 
    data_pointer = 0 
    
    for b in range(batch_size):
        
        inputs = [char_to_int[ch] for ch in txt_data[data_pointer:data_pointer+sequence_length]]
        targets = [char_to_int[ch] for ch in txt_data[data_pointer+1:data_pointer+sequence_length+1]]      
            
        if (data_pointer+sequence_length+1 >= len(txt_data) and b == batch_size-1):  
            targets.append(char_to_int[" "])   

        loss, ps, hs, xs = forwardprop(inputs, targets, h_prev)
    
        dWxh, dWhh, dWhy, dbh, dby = backprop(ps, inputs, hs, xs) 
        
        for param, dparam, mem in zip([W_xh, W_hh, W_hy, b_h, b_y], 
                                    [dWxh, dWhh, dWhy, dbh, dby], 
                                    [mWxh, mWhh, mWhy, mbh, mby]):
            mem += dparam * dparam 
            param += -learning_rate * dparam / np.sqrt(mem + 1e-8)     
    
        data_pointer += sequence_length 
        
    if i % 100 == 0:
        print ('iter %d, loss: %f' % (i, loss)) 

iter 0, loss: 3.375564
iter 100, loss: 0.011112
iter 200, loss: 0.004815
iter 300, loss: 0.003006
iter 400, loss: 0.002147
iter 500, loss: 0.001654
iter 600, loss: 0.001338
iter 700, loss: 0.001120
iter 800, loss: 0.000961
iter 900, loss: 0.000840
iter 1000, loss: 0.000746
iter 1100, loss: 0.000672
iter 1200, loss: 0.000611
iter 1300, loss: 0.000560
iter 1400, loss: 0.000517
iter 1500, loss: 0.000480
iter 1600, loss: 0.000447
iter 1700, loss: 0.000419
iter 1800, loss: 0.000394
iter 1900, loss: 0.000372
iter 2000, loss: 0.000352
iter 2100, loss: 0.000334
iter 2200, loss: 0.000318
iter 2300, loss: 0.000303
iter 2400, loss: 0.000290
iter 2500, loss: 0.000278
iter 2600, loss: 0.000267
iter 2700, loss: 0.000256
iter 2800, loss: 0.000247
iter 2900, loss: 0.000238
iter 3000, loss: 0.000230
iter 3100, loss: 0.000223
iter 3200, loss: 0.000216
iter 3300, loss: 0.000209
iter 3400, loss: 0.000203
iter 3500, loss: 0.000197
iter 3600, loss: 0.000192
iter 3700, loss: 0.000186
iter 3800, loss: 0.00018

In [ ]:
def predict(test_char, length):
    x = np.zeros((num_chars, 1)) 
    x[char_to_int[test_char]] = 1
    ixes = []
    h = np.zeros((hidden_size,1))

    for t in range(length):
        h = np.tanh(np.dot(W_xh, x) + np.dot(W_hh, h) + b_h) 
        y = np.dot(W_hy, h) + b_y
        p = np.exp(y) / np.sum(np.exp(y)) 
        ix = np.random.choice(range(num_chars), p=p.ravel()) 
        x = np.zeros((num_chars, 1)) 
        x[ix] = 1 
        ixes.append(ix) 
    txt = ''.join(int_to_char[i] for i in ixes)
    print ('----\n %s \n----' % (txt, ))

In [ ]:
predict('a',30)

----
 bcdefghijklmnopqrstulmnopqrstu 
----


In [ ]:
predict('b',30)

----
 wmnefghijklmnopqrstulmnopqrstu 
----
